In [1]:
import pandas as pd

import psycopg2 as pg2
con = pg2.connect(host='this_postgres', user='postgres',database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

Check running processes

In [2]:
sql = '''
SELECT pid , query, *
FROM pg_stat_activity pgsa
WHERE pid <> pg_backend_pid()
'''

In [3]:
df = select(sql)

In [4]:
df

,pid,query,datid,datname,pid,usesysid,usename,application_name,client_addr,client_hostname,...,xact_start,query_start,state_change,wait_event_type,wait_event,state,backend_xid,backend_xmin,query,backend_type
0,63,,NaN,None,63,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,AutoVacuumMain,None,None,None,,autovacuum launcher
1,64,,NaN,None,64,10.0,postgres,,None,None,...,NaT,NaT,NaT,Activity,LogicalLauncherMain,None,None,None,,background worker
2,74,COMMIT,13044.0,postgres,74,10.0,postgres,,172.19.0.2,None,...,NaT,2019-12-29 04:58:11.011453+00:00,2019-12-29 04:58:11.014250+00:00,Client,ClientRead,idle,None,None,COMMIT,client backend
3,75,select * from hello_table t,13044.0,postgres,75,10.0,postgres,,172.19.0.2,None,...,2019-12-29 04:58:11.100961+00:00,2019-12-29 04:58:11.404119+00:00,2019-12-29 04:58:11.404311+00:00,Client,ClientRead,idle in transaction,576,None,select * from hello_table t,client backend
4,47,,NaN,None,47,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,BgWriterHibernate,None,None,None,,background writer
5,46,,NaN,None,46,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,CheckpointerMain,None,None,None,,checkpointer
6,48,,NaN,None,48,NaN,None,,None,None,...,NaT,NaT,NaT,Activity,WalWriterMain,None,None,None,,walwriter


Kill all

In [5]:
sql = '''
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    -- don't kill my own connection!
    pid <> pg_backend_pid()

'''

In [6]:
cur.execute(sql)